# Reproduction of MODFLOW-NWT Problem 3



## MODFLOW-NWT

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import flopy
import flopy.utils.binaryfile as bf

In [ ]:
#paths to existing output
mf6pth = [os.path.join('..', 'test014_NWTP3High'), 
          os.path.join('..', 'test014_NWTP3Low_dev')]
mfnwtpth = [os.path.join('..', 'test014_NWTP3High_dev', 'mfusg'), 
             os.path.join('..', 'test014_NWTP3Low_dev', 'mfnwt')]

In [ ]:
#setup model parameters
xmax = 8000.
ymax = 8000.
delr = 100. 
delc = 100.

nrow = int(ymax / delr)
ncol = int(xmax / delr)

#--x
x = np.arange(0, xmax, delr) + delr/2.
#--y
y = np.arange(0, ymax, delr) + delc/2.
#--mesh grid object
X, Y = np.meshgrid(x, y[::-1])

### Get base model

In [ ]:
model = 'NWTP3.nam'
ml = flopy.modflow.Modflow.load(model, version='mfnwt', model_ws=mfnwtpth[0], verbose=True)

#### set aquifer bottom

In [ ]:
aqbottom = ml.dis.botm.array[0, :, :]

### High recharge rate

In [ ]:
#--Create the headfile objects
#--MODFLOW-USG
headfile = os.path.join(mfnwtpth[0], 'NWTP3.hds')
nwtobj = bf.HeadFile(headfile, precision='double')
times = nwtobj.get_times()
head = nwtobj.get_data(totim=times[-1]) 
nwthead = head[-1, :, :]
nwtsat = nwthead - aqbottom[:, :]
for irow in range(nrow):
    for jcol in range(ncol):
        if nwthead[irow, jcol] < aqbottom[irow, jcol]+0.02 or nwthead[irow, jcol] < -998.:
            nwthead[irow, jcol] = np.nan
            nwtsat[irow, jcol] = np.nan
            
#--MODFLOW 6
headfile = os.path.join(mf6pth[0], 'nwtp3.hds')
mf6obj = bf.HeadFile(headfile, precision='double')
times = nwtobj.get_times()
head = mf6obj.get_data(totim=times[-1]) 
mf6head = head[-1, :, :]
mf6sat = mf6head - aqbottom[:, :]
for irow in range(nrow):
    for jcol in range(ncol):
        if mf6head[irow, jcol] < aqbottom[irow, jcol]+0.02 or mf6head[irow, jcol] < -998.:
            mf6head[irow, jcol] = np.nan
            mf6sat[irow, jcol] = np.nan 
print((mf6head-nwthead).max())

### Simulated head

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1, 3, 1)
h = ax.imshow(nwthead, cmap='jet', extent=[0, xmax, 0., ymax], vmin=20, vmax=80, origin='upper', interpolation='None')
c = ax.contour(X, Y, nwthead, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW-USG', size=8)

ax = fig.add_subplot(1, 3, 2)
h = ax.imshow(mf6head, cmap='jet', extent=[0, xmax, 0., ymax], vmin=20, vmax=80, origin='upper', interpolation='None')
c = ax.contour(X, Y, mf6head, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW 6', size=8)

diff = 100. * (mf6head - nwthead) / nwthead
ax = fig.add_subplot(1, 3, 3)
h = ax.imshow(diff, cmap='jet', extent=[0, xmax, 0., ymax], origin='upper', interpolation='None')
c = ax.contour(X, Y, diff, colors='k')
ax.clabel(c, fmt='%3.0e')
plt.colorbar(h, shrink=0.5)
plt.title('Percent Difference', size=8);

### Saturated thickness

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1, 3, 1)
h = ax.imshow(nwtsat, cmap='jet', extent=[0, xmax, 0., ymax], vmin=0, vmax=30, origin='upper', interpolation='None')
c = ax.contour(X, Y, nwtsat, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW-USG', size=8)

ax = fig.add_subplot(1, 3, 2)
h = ax.imshow(mf6sat, cmap='jet', extent=[0, xmax, 0., ymax], vmin=0, vmax=30, origin='upper', interpolation='None')
c = ax.contour(X, Y, mf6sat, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW 6', size=8)

diff = 100. * (mf6sat - nwtsat) / nwtsat
ax = fig.add_subplot(1, 3, 3)
h = ax.imshow(diff, cmap='jet', extent=[0, xmax, 0., ymax], origin='upper', interpolation='None')
c = ax.contour(X, Y, diff, colors='k')
ax.clabel(c, fmt='%3.0e')
plt.colorbar(h, shrink=0.5)
plt.title('Percent Difference', size=8);

### Low recharge rate

In [ ]:
#--Create the headfile objects
#--MODFLOW-NWT
headfile = os.path.join(mfnwtpth[1], 'NWTP3.hds')
nwtobj = bf.HeadFile(headfile, precision='double')
times = nwtobj.get_times()
head = nwtobj.get_data(totim=times[-1]) 
nwthead = head[-1, :, :]
nwtsat = nwthead - aqbottom[:, :]

            
#--MODFLOW 6
headfile = os.path.join(mf6pth[1], 'nwtp3.hds')
mf6obj = bf.HeadFile(headfile, precision='double')
times = nwtobj.get_times()
head = mf6obj.get_data(totim=times[-1]) 
mf6head = head[-1, :, :]
mf6sat = mf6head - aqbottom[:, :]

In [ ]:
d = mf6head - nwthead
print(d.min(), d.max(), d.mean(), d.std())

In [ ]:
# fill "dry" cells with np.nan
for irow in range(nrow):
    for jcol in range(ncol):
        if nwthead[irow, jcol] < aqbottom[irow, jcol]+0.02 or nwthead[irow, jcol] < -998.:
            nwthead[irow, jcol] = np.nan
            nwtsat[irow, jcol] = np.nan
            
for irow in range(nrow):
    for jcol in range(ncol):
        if mf6head[irow, jcol] < aqbottom[irow, jcol]+0.02 or mf6head[irow, jcol] < -998.:
            mf6head[irow, jcol] = np.nan
            mf6sat[irow, jcol] = np.nan            

### Simulated head

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1, 3, 1)
h = ax.imshow(nwthead, cmap='jet', extent=[0, xmax, 0., ymax], vmin=20, vmax=80, origin='upper', interpolation='None')
c = ax.contour(X, Y, nwthead, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW-NWT', size=8)

ax = fig.add_subplot(1, 3, 2)
h = ax.imshow(mf6head, cmap='jet', extent=[0, xmax, 0., ymax], vmin=20, vmax=80, origin='upper', interpolation='None')
c = ax.contour(X, Y, mf6head, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW 6', size=8)

diff = 100. * (mf6head - nwthead) / nwthead
ax = fig.add_subplot(1, 3, 3)
h = ax.imshow(diff, cmap='jet', extent=[0, xmax, 0., ymax], origin='upper', interpolation='None')
c = ax.contour(X, Y, diff, colors='k')
ax.clabel(c, fmt='%3.0e')
plt.colorbar(h, shrink=0.5)
plt.title('Percent Difference', size=8);

### Saturated thickness

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1, 3, 1)
h = ax.imshow(nwtsat, cmap='jet', extent=[0, xmax, 0., ymax], vmin=0, vmax=30, origin='upper', interpolation='None')
c = ax.contour(X, Y, nwtsat, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW-NWT', size=8)

ax = fig.add_subplot(1, 3, 2)
h = ax.imshow(mf6sat, cmap='jet', extent=[0, xmax, 0., ymax], vmin=0, vmax=30, origin='upper', interpolation='None')
c = ax.contour(X, Y, mf6sat, colors='k', levels=np.arange(0,100,5))
ax.clabel(c, fmt='%1.0f')
plt.colorbar(h, shrink=0.5)
plt.title('MODFLOW 6', size=8)

diff = 100. * (mf6sat - nwtsat) / nwtsat
ax = fig.add_subplot(1, 3, 3)
h = ax.imshow(diff, cmap='jet', extent=[0, xmax, 0., ymax], origin='upper', interpolation='None')
c = ax.contour(X, Y, diff, colors='k')
ax.clabel(c, fmt='%3.0e')
plt.colorbar(h, shrink=0.5)
plt.title('Percent Difference', size=8);